In [ ]:
%pip install -r requirements.txt
# Install required packages from 'requirements.txt' (if necessary).

In [2]:
from langchain_openai import OpenAI
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [4]:
## Data Ingestion
loader=TextLoader("data.txt", encoding="utf-8")
text_documents=loader.load()
text_documents

[Document(metadata={'source': 'data.txt'}, page_content='\nEconomics\nTOPICS\nOUTLOOKS\nRESEARCH CENTERS\n\n\nFor You\nIndia economic outlook, January 2025\nAmid slowing GDP growth, demographic dividends and growing middle-class wealth are driving India’s resilience in consumption, services, and more importantly, capital markets\n\nArticle\n • \n11-min read\n • \n16 January 2025\n • \nDeloitte Global Economics Research Center\nDr. Rumki Majumdar\nDr. Rumki Majumdar\nIndia\nDebdatta Ghatak\nDebdatta Ghatak\nIndia\nThe recent GDP growth figures of 5.4% year over year1 for the second quarter of fiscal year 2024 to 20252 probably caught markets off guard (it was significantly below the Reserve Bank of India’s projection of 6.8%). Slower growth in the first half of the fiscal (6%) led the RBI to bring down the annual projection to 6.6% (down from an earlier projection of 7%). However, it’s essential not to let the headline numbers overshadow the nuanced story beneath: GDP is just one lens t

In [5]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200,length_function=len)
documents=text_splitter.split_documents(text_documents)
documents[0]

Document(metadata={'source': 'data.txt'}, page_content='Economics\nTOPICS\nOUTLOOKS\nRESEARCH CENTERS\n\n\nFor You\nIndia economic outlook, January 2025\nAmid slowing GDP growth, demographic dividends and growing middle-class wealth are driving India’s resilience in consumption, services, and more importantly, capital markets')

In [6]:
len(documents)


35

In [7]:
embedding = OpenAIEmbeddings()

In [8]:
vector_store = FAISS.from_documents(documents, embedding=embedding)
vector_store.save_local("faiss_index_india_econ")

In [9]:
Query1 = "What is India's projected GDP growth for the fiscal year 2024-2025?"

In [10]:
Query_Answer = vector_store.similarity_search(Query1)

In [11]:
print(Query_Answer[0].page_content)

India’s near-term outlook
We now expect India to grow between 6.5% and 6.8% in fiscal year 2024 to 2025, in our baseline scenario. Although admittedly lower than previously estimated, because of a slower first half of the year, we expect strong domestic demand in the second half, driven by a significant uptick in government spending).


In [12]:
docs_and_scores = vector_store.similarity_search_with_score(Query1)

In [13]:
docs_and_scores[0]

(Document(metadata={'source': 'data.txt'}, page_content='India’s near-term outlook\nWe now expect India to grow between 6.5% and 6.8% in fiscal year 2024 to 2025, in our baseline scenario. Although admittedly lower than previously estimated, because of a slower first half of the year, we expect strong domestic demand in the second half, driven by a significant uptick in government spending).'),
 0.15318276)

In [14]:
retriever = vector_store.as_retriever()

In [15]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)

In [16]:
retriever_query1 = "What is India's projected GDP growth for the fiscal year 2024-2025?"

results = qa.invoke(retriever_query1)
print(results)

{'query': "What is India's projected GDP growth for the fiscal year 2024-2025?", 'result': " India's projected GDP growth for the fiscal year 2024-2025 is between 6.5% and 6.8%."}


In [17]:
retriever_query2 = "What factors are contributing to the expected strong domestic demand in the second half of the fiscal year?"

results = qa.invoke(retriever_query2)
print(results)

{'query': 'What factors are contributing to the expected strong domestic demand in the second half of the fiscal year?', 'result': ' A significant uptick in government spending and crowding in private investments.'}


In [18]:
retriever_query3 = "What is the projected GDP growth for the fiscal year 2025-2026?"

results = qa.invoke(retriever_query3)
print(results)

{'query': 'What is the projected GDP growth for the fiscal year 2025-2026?', 'result': ' The projected GDP growth for the fiscal year 2025-2026 is between 6.7% and 7.3%.'}


In [19]:
retriever_query4 = "What are the significant downside risks to India's economic growth outlook?"

results = qa.invoke(retriever_query4)
print(results)

{'query': "What are the significant downside risks to India's economic growth outlook?", 'result': " The significant downside risks to India's economic growth outlook include potential inflation concerns, disruptions to global trade and supply chain, and geopolitical uncertainties affecting demand for exports."}


In [20]:
retriever_query5 = "How have geopolitical uncertainties affected India's economic projections?"

results = qa.invoke(retriever_query5)
print(results)

{'query': "How have geopolitical uncertainties affected India's economic projections?", 'result': " Geopolitical uncertainties have led to significant downside risks in India's economic projections, resulting in a wider range of growth projections and a potential delay in global recovery. They have also caused disruptions to global trade and supply chains, potentially affecting demand for Indian exports. Additionally, changes in policy and political stability in advanced economies may have limited implications on trade and investments, which could impact India's economic growth."}


In [21]:
retriever_query6 = "What impact could disruptions to global trade and supply chains have on India's economy?"

results = qa.invoke(retriever_query6)
print(results)

{'query': "What impact could disruptions to global trade and supply chains have on India's economy?", 'result': " Disruptions to global trade and supply chains could potentially affect demand for Indian exports and weigh on India's economic growth."}


In [22]:
retriever_query7 = "How does the current growth projection compare to previous estimates?"

results = qa.invoke(retriever_query7)
print(results)

{'query': 'How does the current growth projection compare to previous estimates?', 'result': ' The current growth projection is lower than previous estimates, with a range of 6.7% to 7.3% in fiscal year 2025 to 2026. This is due to significant downside risks and potential disruptions to global trade and supply chain.'}


In [23]:
retriever_query8 = "What role does government spending play in India's economic outlook?"

results = qa.invoke(retriever_query8)
print(results)

{'query': "What role does government spending play in India's economic outlook?", 'result': ' Government spending can play a significant role in boosting domestic demand and supporting economic growth in India. The government may prioritize strengthening domestic demand, particularly by focusing on boosting consumer spending, in order to drive sustainable growth. However, government spending must be carefully managed to balance the potential risks of inflation and maintain fiscal stability.'}


In [24]:
retriever_query9 = "What are the implications of a delayed synchronous recovery in Western economies for India?"

results = qa.invoke(retriever_query9)
print(results)

{'query': 'What are the implications of a delayed synchronous recovery in Western economies for India?', 'result': " The delayed synchronized recovery in Western economies could potentially weigh on India's exports and outlook for the next fiscal year. This could also result in a slower pace of growth for India and could require the country to adapt to the evolving global landscape and focus on harnessing its domestic strengths to drive sustainable growth."}


In [25]:
retriever_query10 = "What measures can India take to mitigate the risks associated with geopolitical uncertainties and global trade disruptions?"

results = qa.invoke(retriever_query10)
print(results)

{'query': 'What measures can India take to mitigate the risks associated with geopolitical uncertainties and global trade disruptions?', 'result': ' India can focus on strengthening domestic institutional investment and increasing its middle-class wealth to boost consumption and stabilize its financial markets. It can also monitor and regulate imports from neighboring countries to prevent dumping and protect its domestic industries. Additionally, India can look for alternative trade partners and diversify its export markets to mitigate the impact of trade disruptions in industrialized nations. '}
